In [23]:
import damask
import numpy as np
import yaml

# Example: Generate single phase single material polycrystal material config

In [24]:
with open('./examples/config/phase/mechanical/elastic/Hooke_al.yaml', 'r') as f:
    al_elastic = yaml.safe_load(f)
with open('./examples/config/phase/mechanical/plastic/phenopowerlaw_Al.yaml', 'r') as f:
    al_plastic = yaml.safe_load(f)

In [25]:
mat = damask.ConfigMaterial()

homogenization = {
    'SX': {
        'N_constituents': 1,
        'mechanical': {'type': 'pass'} # No volume fractions, so no homogenization needed
    }
}
mat.update(homogenization=homogenization)

phase_alu = {
    'Aluminium':{
        'lattice': 'cF', # FCC material
        'mechanical': {
            'output': ['F', 'P', 'F_e', 'L_p', 'O'],
            'elastic': al_elastic,
            'plastic': al_plastic
        }
    }
}
mat.update(phase=phase_alu)

N_grains = 10
rotations = damask.Rotation.from_random(N_grains)
mat = mat.material_add(O=rotations,phase='Aluminium',homogenization='SX')
mat.is_complete

True

In [26]:
mat.save(f'./material_fullfield_{N_grains}grains_Al.yaml')

# Example: Generate DP Steel with distinct grains

In [27]:
with open('./examples/config/phase/mechanical/elastic/Hooke_Fe.yaml', 'r') as f:
    martensite_elastic = yaml.safe_load(f)
with open('./examples/config/phase/mechanical/plastic/phenopowerlaw_bcc-martensite.yaml', 'r') as f:
    martensite_plastic = yaml.safe_load(f)

with open('./examples/config/phase/mechanical/elastic/Hooke_Fe.yaml', 'r') as f:
    ferrite_elastic = yaml.safe_load(f)
with open('./examples/config/phase/mechanical/plastic/phenopowerlaw_DP-steel-ferrite.yaml', 'r') as f:
    ferrite_plastic = yaml.safe_load(f)

In [28]:
mat = damask.ConfigMaterial()

homogenization = {
    'SX': {
        'N_constituents': 1,
        'mechanical': {'type': 'pass'} # No volume fractions, so no homogenization needed
    }
}
mat.update(homogenization=homogenization)

phase_martesite = {
    'Martensite':{
        'lattice': 'cI', # BCC
        'mechanical': {
            'output': ['F', 'P', 'F_e', 'L_p', 'O'],
            'elastic': martensite_elastic,
            'plastic': martensite_plastic
        }
    }
}

phase_ferrite = {
    'Ferrite':{
        'lattice': 'cI', # BCC
        'mechanical': {
            'output': ['F', 'P', 'F_e', 'L_p', 'O'],
            'elastic': ferrite_elastic,
            'plastic': ferrite_plastic
        }
    }
}

mat.update(phase=phase_martesite|phase_ferrite) # merge dictionaries

N_grains_ferrite = 6
rotations = damask.Rotation.from_random(N_grains_ferrite)
mat = mat.material_add(O=rotations,phase='Ferrite',homogenization='SX')

N_grains_martensite = 4
rotations = damask.Rotation.from_random(N_grains_martensite)
mat = mat.material_add(O=rotations,phase='Martensite',homogenization='SX')

mat.is_complete

True

In [29]:
mat.save(f'material_fullfield_{N_grains_ferrite+N_grains_martensite}materials_{N_grains_ferrite}ferrite_{N_grains_martensite}martensite.yaml')

# Example: Generate DP steel with volume fraction

In [30]:
with open('./examples/config/phase/mechanical/elastic/Hooke_Fe.yaml', 'r') as f:
    martensite_elastic = yaml.safe_load(f)
with open('./examples/config/phase/mechanical/plastic/phenopowerlaw_bcc-martensite.yaml', 'r') as f:
    martensite_plastic = yaml.safe_load(f)

with open('./examples/config/phase/mechanical/elastic/Hooke_Fe.yaml', 'r') as f:
    ferrite_elastic = yaml.safe_load(f)
with open('./examples/config/phase/mechanical/plastic/phenopowerlaw_DP-steel-ferrite.yaml', 'r') as f:
    ferrite_plastic = yaml.safe_load(f)

In [31]:
N_materials = 10
martensite_fraction = 0.4
ferrite_fraction = 0.6

mat = damask.ConfigMaterial()

homogenization = {
    'Taylor': {
        'N_constituents': 2,
        'mechanical': {'type': 'isostrain'} # Volume fractions present, so homogenization needed
    }
}
mat.update(homogenization=homogenization)

phase_martesite = {
    'Martensite':{
        'lattice': 'cI', # BCC
        'mechanical': {
            'output': ['F', 'P', 'F_e', 'L_p', 'O'],
            'elastic': martensite_elastic,
            'plastic': martensite_plastic
        }
    }
}

phase_ferrite = {
    'Ferrite':{
        'lattice': 'cI', # BCC
        'mechanical': {
            'output': ['F', 'P', 'F_e', 'L_p', 'O'],
            'elastic': ferrite_elastic,
            'plastic': ferrite_plastic
        }
    }
}

mat.update(phase=phase_martesite|phase_ferrite)

mat = mat.material_add(phase = ['Martensite', 'Ferrite'],
                       O = damask.Rotation.from_random((N_materials,2)),
                       v = np.array([[martensite_fraction]+[ferrite_fraction]]),
                       homogenization = 'Taylor')
mat.is_complete

True

In [32]:
mat.save(f'material_meanfield_{N_materials}materials_{ferrite_fraction}ferrite_{martensite_fraction}martensite.yaml')

## Exercises:

- Generate a single-phase IF-steel material with 50 distinct materials (grains), using the dislotwin constitutive law.
  - Bonus: generate a corresponding gridfile and run the simulation
- Generate a dual-phase steel with 20 distinct materials and a volume fraction of 50% each

In [33]:
with open('./examples/config/phase/mechanical/elastic/Hooke_Fe.yaml', 'r') as f:
    fe_elastic = yaml.safe_load(f)
with open('./examples/config/phase/mechanical/plastic/dislotwin_IF-steel.yaml', 'r') as f:
    fe_plastic = yaml.safe_load(f)

mat = damask.ConfigMaterial()

homogenization = {
    'SX': {
        'N_constituents': 1,
        'mechanical': {'type': 'pass'} # No volume fractions, so no homogenization needed
    }
}
mat.update(homogenization=homogenization)

phase_alu = {
    'IFSteel':{
        'lattice': 'cI', # BCC material
        'mechanical': {
            'output': ['F', 'P', 'F_e', 'L_p', 'O'],
            'elastic': fe_elastic,
            'plastic': fe_plastic
        }
    }
}
mat.update(phase=phase_alu)

N_grains = 50
rotations = damask.Rotation.from_random(N_grains)
mat = mat.material_add(O=rotations,phase='IFSteel',homogenization='SX')
mat.save(f'material_fullfield_{N_grains}grains_IFSteel.yaml')


In [34]:
with open('./examples/config/phase/mechanical/elastic/Hooke_Fe.yaml', 'r') as f:
    martensite_elastic = yaml.safe_load(f)
with open('./examples/config/phase/mechanical/plastic/phenopowerlaw_bcc-martensite.yaml', 'r') as f:
    martensite_plastic = yaml.safe_load(f)

with open('./examples/config/phase/mechanical/elastic/Hooke_Fe.yaml', 'r') as f:
    ferrite_elastic = yaml.safe_load(f)
with open('./examples/config/phase/mechanical/plastic/phenopowerlaw_DP-steel-ferrite.yaml', 'r') as f:
    ferrite_plastic = yaml.safe_load(f)

N_materials = 20
martensite_fraction = 0.5
ferrite_fraction = 0.5

mat = damask.ConfigMaterial()

homogenization = {
    'Taylor': {
        'N_constituents': 2,
        'mechanical': {'type': 'isostrain'} # Volume fractions present, so homogenization needed
    }
}
mat.update(homogenization=homogenization)

phase_martesite = {
    'Martensite':{
        'lattice': 'cI', # BCC
        'mechanical': {
            'output': ['F', 'P', 'F_e', 'L_p', 'O'],
            'elastic': martensite_elastic,
            'plastic': martensite_plastic
        }
    }
}

phase_ferrite = {
    'Ferrite':{
        'lattice': 'cI', # BCC
        'mechanical': {
            'output': ['F', 'P', 'F_e', 'L_p', 'O'],
            'elastic': ferrite_elastic,
            'plastic': ferrite_plastic
        }
    }
}

mat.update(phase=phase_martesite|phase_ferrite)

mat = mat.material_add(phase = ['Martensite', 'Ferrite'],
                       O = damask.Rotation.from_random((N_materials,2)),
                       v = np.array([[martensite_fraction]+[ferrite_fraction]]),
                       homogenization = 'Taylor')
mat.save(f'material_meanfield_{N_materials}materials_{ferrite_fraction}ferrite_{martensite_fraction}martensite.yaml')